In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv("dataset_full_big3.csv")

In [3]:
df['alzheimer'].value_counts()

0    677
1     52
2     41
Name: alzheimer, dtype: int64

In [4]:
df

,Unnamed: 0,code,alzheimer,age,education_x,ws_big_big3_pauses,ws_big_big3_pauses_long,ws_big_big3_pauses_short,ws_big_big3_uniquewords,ws_big_big3_uniquewordsnorm,...,big3,big3_2,big3_pauses,big3_tags,big3_lema,big3_2_tags,big3_pauses_tags,big3_first_tag,big3_2_first_tag,big3_pauses_first_tag
0,0,3fiigiv401,0,52,2,25,17,8,81,81,...,a muž v kuchyni varí a má aj pomocníka ten doč...,a muž v kuchyni varí a má aj pomocníka [spk] [...,[LONGSIL] a muž v kuchyni varí a má aj pomocní...,O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T SSms4 PFm...,a|s muž v kuchyňa variť a|s mať aj pomocník t...,O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T SSms4 xsp...,xlongsilx O SSms1 Eu6 SSfs6 VKesc+ O VKesc+ T...,OOOO SSSS EEEE SSSS VVVV OOOO VVVV TTTT SSSS ...,OOOO SSSS EEEE SSSS VVVV OOOO VVVV TTTT SSSS ...,xlongsilx OOOO SSSS EEEE SSSS VVVV OOOO VVVV ...
1,1,ysw9a5m701,0,66,2,23,9,14,97,97,...,a ja aj toto je toto je by mohol nevie dal ale...,a ja aj toto je toto je by mohol nevie dal ale...,[LONGSIL] a ja aj toto je toto je by [SHORTSIL...,O PPhs1 T PFns1 VKesc+ PFns1 VKesc+ Y VLesci+...,a|s ja aj toto byť|jesť toto byť|jesť by môcť...,O PPhs1 T PFns1 VKesc+ PFns1 VKesc+ Y VLesci+...,xlongsilx O PPhs1 T PFns1 VKesc+ PFns1 VKesc+...,OOOO PPPP TTTT PPPP VVVV PPPP VVVV YYYY VVVV ...,OOOO PPPP TTTT PPPP VVVV PPPP VVVV YYYY VVVV ...,xlongsilx OOOO PPPP TTTT PPPP VVVV PPPP VVVV ...
2,2,lpvvez2301,2,70,2,17,5,12,63,63,...,niekto tam buchol do tej do toho lustra dieťa ...,niekto tam buchol do tej [fil] do toho lustra ...,[LONGSIL] niekto tam buchol do tej [fil] do to...,PFms1 PD VLdscm+ Eu2 PFfs2 Eu2 PFis2 SSis2 SS...,niekto tam buchnúť do tá do ten luster dieťa ...,PFms1 PD VLdscm+ Eu2 PFfs2 xfilx Eu2 PFis2 SS...,xlongsilx PFms1 PD VLdscm+ Eu2 PFfs2 xfilx Eu...,PPPP PPPP VVVV EEEE PPPP EEEE PPPP SSSS SSSS ...,PPPP PPPP VVVV EEEE PPPP xfilx EEEE PPPP SSSS...,xlongsilx PPPP PPPP VVVV EEEE PPPP xfilx EEEE...
3,3,lpvvweke01,0,71,3,24,17,7,39,39,...,kuchyňa otec so synom chlapec sa snaží pomôcť ...,kuchyňa otec so synom chlapec sa snaží pomôcť ...,[LONGSIL] kuchyňa [LONGSIL] otec so synom [LON...,SSfs1 SSms1 Ev7 SSms7 SSms1 R VKesc+ VId+ SSm...,kuchyňa otec s syn chlapec sa snažiť pomôcť o...,SSfs1 SSms1 Ev7 SSms7 SSms1 R VKesc+ VId+ SSm...,xlongsilx SSfs1 xlongsilx SSms1 Ev7 SSms7 xlo...,SSSS SSSS EEEE SSSS SSSS RRRR VVVV VVVV SSSS ...,SSSS SSSS EEEE SSSS SSSS RRRR VVVV VVVV SSSS ...,xlongsilx SSSS xlongsilx SSSS EEEE SSSS xlong...
4,4,fgl9b8ym01,0,87,2,35,14,21,62,62,...,že no červený manžel no tak najsamprv malý zle...,že no červený manžel [@] no tak najsamprv malý...,[LONGSIL] že [SHORTSIL] no [SHORTSIL] červený ...,O O AAms1x SSms1 O T Dx AAis1x Dx VLdscm+ Eu4...,že no červený manžel no tak najsamprv malý zl...,O O AAms1x SSms1 xzavx O PD Dx AAis1x Dx VLds...,xlongsilx O xshortsilx xshortsilx AAms1x SSms...,OOOO OOOO AAAA SSSS OOOO TTTT DDDD AAAA DDDD ...,OOOO OOOO AAAA SSSS xzavx OOOO PPPP DDDD AAAA...,xlongsilx OOOO xshortsilx xshortsilx AAAA SSS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,827,tbc5xe1q01,0,62,1,23,16,7,51,51,...,no tak dobre chlapček stojí na stoličke naťahu...,no tak dobre chlapček stojí na stoličke naťahu...,[LONGSIL] no tak dobre chlapček stojí na stoli...,O PD Dx SSms1 VKesc+ Eu6 SSfs6 VKesc+ R Eu7 S...,no tak dobre chlapček stáť na stolička naťaho...,O PD Dx SSms1 VKesc+ Eu6 SSfs6 VKesc+ R Eu7 S...,xlongsilx O PD Dx SSms1 VKesc+ Eu6 SSfs6 VKes...,OOOO PPPP DDDD SSSS VVVV EEEE SSSS VVVV RRRR ...,OOOO PPPP DDDD SSSS VVVV EEEE SSSS VVVV RRRR ...,xlongsilx OOOO PPPP DDDD SSSS VVVV EEEE SSSS ...
766,828,lpvve13n01,2,71,3,15,11,4,33,33,...,v kuchyni je och otec asi zo synov ktorých zre...,[fil] v kuchyni [fil] je och [@] otec asi zo s...,[LONGSIL] [fil] v kuchyni [LONGSIL] [fil] je o...,Eu6 SSfs6 VKesc+ J SSms1 T Ev2 SSmp2 PAfp2 T ...,v kuchyňa byť|jesť och otec asi z syn ktorý z...,xfilx Eu6 SSfs6 xfilx VKesc+ J xzavx SSms1 T ...,xlongsilx xfilx Eu6 SSfs6 xlongsilx xfilx VKe...,EEEE SSSS VVVV JJJJ SSSS 

In [5]:
vk = CountVectorizer()

In [6]:
y = df['alzheimer']

# Big3

In [7]:
drop = vk.fit_transform(df['big3'].dropna())

In [8]:
vektor = vk.fit_transform(df['big3'])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [10]:
undersampler = RandomUnderSampler()
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [11]:
print(pd.Series(y_train_under).value_counts())

0    33
1    33
2    33
Name: alzheimer, dtype: int64


In [12]:
clf_under = RandomForestClassifier(n_estimators=100, class_weight="balanced")
clf_under.fit(X_train_under, y_train_under)
y_pred_under = clf_under.predict(X_test)

In [13]:
print("Undersampling")
print(f'Presnosť modelu: {accuracy_score(y_test, y_pred_under):.2f}')
print('')
print(classification_report(y_test, y_pred_under, zero_division=1))

Undersampling
Presnosť modelu: 0.58

              precision    recall  f1-score   support

           0       0.97      0.57      0.72       134
           1       0.11      0.42      0.18        12
           2       0.26      1.00      0.41         8

    accuracy                           0.58       154
   macro avg       0.45      0.66      0.44       154
weighted avg       0.87      0.58      0.66       154



In [14]:
oversampler = SMOTE()
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

In [15]:
print(pd.Series(y_train_over).value_counts())

0    543
1    543
2    543
Name: alzheimer, dtype: int64


In [16]:
clf_over = RandomForestClassifier(n_estimators=100, class_weight="balanced")
clf_over.fit(X_train_over, y_train_over)
y_pred_over = clf_over.predict(X_test)

In [17]:
print("SMOTE")
print(f'Presnosť modelu: {accuracy_score(y_test, y_pred_over):.2f}')
print('')
print(classification_report(y_test, y_pred_over, zero_division=1))

SMOTE
Presnosť modelu: 0.88

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       134
           1       0.20      0.17      0.18        12
           2       0.75      0.38      0.50         8

    accuracy                           0.88       154
   macro avg       0.63      0.50      0.54       154
weighted avg       0.86      0.88      0.87       154



In [18]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
y_pred = clf.predict(X_test)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.88

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       134
           1       0.00      0.00      0.00        12
           2       1.00      0.12      0.22         8

    accuracy                           0.88       154
   macro avg       0.63      0.38      0.39       154
weighted avg       0.81      0.88      0.82       154



C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Big3_pauses_first_tag

In [21]:
drop = vk.fit_transform(df['big3_pauses_first_tag'].dropna())

In [22]:
vektor = vk.fit_transform(df['big3_pauses_first_tag'])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [24]:
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred = clf.predict(X_test)

In [26]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.86

              precision    recall  f1-score   support

           0       0.87      0.98      0.93       132
           1       0.00      0.00      0.00        12
           2       0.40      0.20      0.27        10

    accuracy                           0.86       154
   macro avg       0.42      0.39      0.40       154
weighted avg       0.77      0.86      0.81       154



C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Test

In [27]:
vektor = vk.fit_transform(df['big3_pauses_first_tag'])

In [28]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [29]:
clf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
clf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced')

In [30]:
y_pred = clf.predict(X_test)

In [31]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.89

              precision    recall  f1-score   support

           0       0.89      0.99      0.94       137
           1       0.00      0.00      0.00        10
           2       0.50      0.14      0.22         7

    accuracy                           0.89       154
   macro avg       0.46      0.38      0.39       154
weighted avg       0.82      0.89      0.85       154



C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Logistická

In [32]:
vektor = vk.fit_transform(df['big3_pauses_first_tag'])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [34]:
clf = LogisticRegression(class_weight="balanced", max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [35]:
y_pred = clf.predict(X_test)

In [36]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.68

              precision    recall  f1-score   support

           0       0.99      0.69      0.82       140
           1       0.16      0.50      0.24        10
           2       0.12      0.75      0.21         4

    accuracy                           0.68       154
   macro avg       0.42      0.65      0.42       154
weighted avg       0.91      0.68      0.76       154



# Ďaľšie

In [37]:
vektor = vk.fit_transform(df['big3'])

In [38]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [39]:
clf = LogisticRegression(class_weight="balanced", max_iter=1000)
clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [40]:
y_pred = clf.predict(X_test)

In [41]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Presnosť modelu: {accuracy:.2f}')
print('')
print(classification_report(y_test, y_pred))

Presnosť modelu: 0.89

              precision    recall  f1-score   support

           0       0.92      0.96      0.94       135
           1       0.17      0.14      0.15         7
           2       0.86      0.50      0.63        12

    accuracy                           0.89       154
   macro avg       0.65      0.54      0.58       154
weighted avg       0.88      0.89      0.88       154



# K-Means

In [42]:
vektor = vk.fit_transform(df['big3'])

In [43]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [44]:
k = len(set(y))
kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(X_train)

KMeans(n_clusters=3, n_init=10)

In [45]:
y_pred = kmeans.predict(X_test)

In [46]:
cluster_map = {}
for i in range(k):
    cluster_labels = y_train[kmeans.labels_ == i]
    if len(cluster_labels) > 0:
        cluster_map[i] = mode(cluster_labels).mode[0]

<ipython-input-46-fb05840046d0>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  cluster_map[i] = mode(cluster_labels).mode[0]


In [47]:
y_pred_mapped = np.array([cluster_map[cluster] for cluster in y_pred])

In [48]:
accuracy = accuracy_score(y_test, y_pred_mapped)
print(f'Presnosť modelu: {accuracy:.2f}')
print(classification_report(y_test, y_pred_mapped))

Presnosť modelu: 0.88
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       136
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         9

    accuracy                           0.88       154
   macro avg       0.29      0.33      0.31       154
weighted avg       0.78      0.88      0.83       154



C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
oversampler = SMOTE()
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

In [50]:
k = len(set(y))
kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(X_train_over)

KMeans(n_clusters=3, n_init=10)

In [51]:
y_pred = kmeans.predict(X_test)

In [52]:
cluster_map = {}
for i in range(k):
    cluster_labels = y_train_over[kmeans.labels_ == i]
    if len(cluster_labels) > 0:
        cluster_map[i] = mode(cluster_labels).mode[0]

<ipython-input-52-61404ac1155f>:5: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  cluster_map[i] = mode(cluster_labels).mode[0]


In [53]:
y_pred_mapped = np.array([cluster_map[cluster] for cluster in y_pred])

In [54]:
accuracy = accuracy_score(y_test, y_pred_mapped)
print(f'Presnosť modelu: {accuracy:.2f}')
print(classification_report(y_test, y_pred_mapped))

Presnosť modelu: 0.75
              precision    recall  f1-score   support

           0       0.95      0.79      0.86       136
           1       0.00      0.00      0.00         9
           2       0.20      0.89      0.32         9

    accuracy                           0.75       154
   macro avg       0.38      0.56      0.39       154
weighted avg       0.85      0.75      0.78       154



C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dgd\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Lineárna regresia

In [55]:
vektor = vk.fit_transform(df['big3'])

In [56]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [57]:
clf = LinearRegression()
clf.fit(X_train, y_train)

LinearRegression()

In [58]:
y_pred = clf.predict(X_test)

In [59]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [60]:
print(f'Presnosť: {r2:.2f}')
print(f'MAE: {mae:.2f}')
print(f'MSE: {mse:.2f}')

Presnosť: -1.56
MAE: 0.58
MSE: 0.53


# Naivný bayesovský model

In [61]:
vektor = vk.fit_transform(df['big3'])

In [62]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [63]:
clf_nb = MultinomialNB()
clf_nb.fit(X_train, y_train)
y_pred_nb = clf_nb.predict(X_test)

In [64]:
print(f'Presnosť modelu: {accuracy_score(y_test, y_pred_nb):.2f}')
print(classification_report(y_test, y_pred_nb, zero_division=1))

Presnosť modelu: 0.88
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       136
           1       0.00      0.00      1.00         9
           2       1.00      0.00      0.00         9

    accuracy                           0.88       154
   macro avg       0.63      0.33      0.64       154
weighted avg       0.84      0.88      0.88       154



# Podporný vektorový klasifikátor

In [65]:
vektor = vk.fit_transform(df['big3'])

In [66]:
X_train, X_test, y_train, y_test = train_test_split(vektor, y, test_size=0.2)

In [67]:
clf_svc = SVC(class_weight="balanced", probability=True)
clf_svc.fit(X_train, y_train)
y_pred_svc = clf_svc.predict(X_test)

In [68]:
print(f'Presnosť modelu: {accuracy_score(y_test, y_pred_svc):.2f}')
print(classification_report(y_test, y_pred_svc, zero_division=1))

Presnosť modelu: 0.82
              precision    recall  f1-score   support

           0       0.93      0.89      0.91       135
           1       0.00      0.00      1.00         8
           2       0.58      0.64      0.61        11

    accuracy                           0.82       154
   macro avg       0.50      0.51      0.84       154
weighted avg       0.86      0.82      0.89       154

